<a href="https://colab.research.google.com/github/rnlima8/DSWP/blob/master/Notebooks/NB10_04__3DP_2_Missing_Value_Handling_RNL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b><i>3DP_2 - MISSING VALUES HANDLING</i></b></h1></center>


# **AGENDA**:

> Consulte **Table of contents**.


___
# **REFERÊNCIAS**
* [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)
* [Handling Missing Data for a Beginner](https://towardsdatascience.com/handling-missing-data-for-a-beginner-6d6f5ea53436)

___
# **3DP_MISSING VALUES HANDLING**

> Lidar com Missing Values é um dos piores pesadelos de um Cientista de dados. Especialmente, se o número de MV for grande o suficiente (geralmente acima de 5%). Nesse caso, os valores não podem ser descartados e um Cientista de Dados inteligente deve "imputar" os valores ausentes.

* Nesta sessão, vamos identificar, analisar e tratar Missing Values (MV).
* Como MV são gerados?
    * Usuário se esqueceu de preencher ou preencheu errado o campo;
    * Os dados foram perdidos durante a transferência manual de um banco de dados legado;
    * Erro de programação;
    * Os usuários optaram por não preencher um campo vinculado a suas crenças sobre como os resultados seriam usados ou interpretados.
* As funções df.isnull() e df.isna() são apropriadas para nos indicar quantas observações são MV no dataframe.

* Na prática:
    * Variáveis Contínuas/Numéricas - Podemos substituir os NaN por Média/Mediana/Moda;
	* Variáveis Categóricas - Uma alternativa é atribuir uma categoria inexistente como, por exemplo "MV" para indicar o NaN.


___
# **MACHINE LEARNING COM PYTHON (Scikit-Learn)**

![Scikit-Learn](https://github.com/MathMachado/Materials/blob/master/scikit-learn-1.png?raw=true)

## Carregar as biliotecas

In [4]:
import pandas as pd
from pandas import Series, DataFrame

import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

## Dataframes
* O dataframe abaixo foi gerado aleatoriamente para entendermos como lidar com os NaN's.

In [5]:
df= pd.DataFrame({
    'idade': [32,38,np.nan,37,np.nan,36,38,32,0,np.nan],
    'salario': ['High', 'High', 'High', 'Low', 'Low', 'High', np.nan, 'Medium', 'Medium', 'High'],
    'pais': ['Spain', 'France', 'France', np.nan, 'Germany', 'France', 'Spain', 'France', np.nan, 'Spain']})

df

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
3,37.0,Low,NaN
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
8,0.0,Medium,NaN
9,NaN,High,Spain


## Identificar os NaN's

A função df.isna() será usada para identificarmos os NaN's nos dataframes. Por exemplo:

In [6]:
df.isna()

,idade,salario,pais
0,False,False,False
1,False,False,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,False,False
6,False,True,False
7,False,False,False
8,False,False,True
9,True,False,False


Qual a interpretação deste output?

Para um dataframe muito grande, vamos usar a expressão abaixo:

In [7]:
df.isna().sum()

idade      3
salario    1
pais       2
dtype: int64

In [8]:
df.isnull()

,idade,salario,pais
0,False,False,False
1,False,False,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,False,False
6,False,True,False
7,False,False,False
8,False,False,True
9,True,False,False


In [9]:
df.isnull().sum()

idade      3
salario    1
pais       2
dtype: int64

In [10]:
df.isnull().count()

idade      10
salario    10
pais       10
dtype: int64

Mais prático não é? No entanto, vamos utilizar a função abaixo, que nos ajudará mais com os NaN's:

In [11]:
def mostra_missing_value(df):  # recebe o data frame como parâmetro
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

In [12]:
mostra_missing_value(df)

         Total  Percentual
idade        3        30.0
pais         2        20.0
salario      1        10.0


## A função df.dropna()
* Esta função deleta as instâncias (linhas do dataframes) onde há pelo menos 1 NaN.

In [13]:
df2 = df.dropna()
df2

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
5,36.0,High,France
7,32.0,Medium,France


Como podemos ver, somente as instâncias 0, 1, 5 e 7 tem atributos não NaN's.

Uma forma menos severa seria:

In [14]:
df3 = df.dropna(axis = 0, subset = ['pais'])
df3

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
9,NaN,High,Spain


* Saberias explicar o que o comando acima fez?

## Tratar os NaN's de Variáveis Numéricas
* Neste exemplo, vou substituir os NaN's da variável 'idade' pela mediana. No entanto, responda a seguinte perfunta:
    * Faz sendido idade= 0?

Acho que a resposta é não. Então, neste caso, 0 é um NaN. Vamos substituído pela mediana da variável:

In [15]:
df['idade2'] = df['idade'].replace({0: df['idade'].median()})
df

,idade,salario,pais,idade2
0,32.0,High,Spain,32.0
1,38.0,High,France,38.0
2,NaN,High,France,NaN
3,37.0,Low,NaN,37.0
4,NaN,Low,Germany,NaN
5,36.0,High,France,36.0
6,38.0,NaN,Spain,38.0
7,32.0,Medium,France,32.0
8,0.0,Medium,NaN,36.0
9,NaN,High,Spain,NaN


Como podemos verificar acima na variável 'idade2', o valor 0 foi substituído pela mediana da variável 'idade'.

Vamos verificar a média da variável antes da operação:

In [16]:
df['idade2'].mean()

35.57142857142857

In [17]:
df['idade3'] = df['idade2']
df

,idade,salario,pais,idade2,idade3
0,32.0,High,Spain,32.0,32.0
1,38.0,High,France,38.0,38.0
2,NaN,High,France,NaN,NaN
3,37.0,Low,NaN,37.0,37.0
4,NaN,Low,Germany,NaN,NaN
5,36.0,High,France,36.0,36.0
6,38.0,NaN,Spain,38.0,38.0
7,32.0,Medium,France,32.0,32.0
8,0.0,Medium,NaN,36.0,36.0
9,NaN,High,Spain,NaN,NaN


Aplicamos a operação:

In [18]:
df['idade3'].fillna(df['idade3'].median(), inplace = True)
df

,idade,salario,pais,idade2,idade3
0,32.0,High,Spain,32.0,32.0
1,38.0,High,France,38.0,38.0
2,NaN,High,France,NaN,36.0
3,37.0,Low,NaN,37.0,37.0
4,NaN,Low,Germany,NaN,36.0
5,36.0,High,France,36.0,36.0
6,38.0,NaN,Spain,38.0,38.0
7,32.0,Medium,France,32.0,32.0
8,0.0,Medium,NaN,36.0,36.0
9,NaN,High,Spain,NaN,36.0


Podemos observar que os valores NaN's do atributo 'idade3' foi substituído pelo valor 36.

E agora, a média após a operação:

In [19]:
df['idade3'].mean()

35.7

* Qual a conclusão?
    * Houve muito impacto na distribuição da variável 'idade'?

## Tratar NaN's de Variáveis Categóricas
* Observe a variável 'pais'. Temos alguns NaN's. As alternativas que temos são:
    * substituir os NaN's desta variável pela moda (valor mais frequente) da distribuição.
    * substiruir os NaN's por 'Undefined'.

Qual o valor (no caso, País) mais frequente ?

In [20]:
df.pais.value_counts()

France     4
Spain      3
Germany    1
Name: pais, dtype: int64

Ok, a instância 'France' é o mais frequente. Então vamos substituir os NaN's por 'France'. De forma automática, temos:

In [21]:
sMode_Of_pais = df['pais'].mode()[0]
sMode_Of_pais

'France'

In [22]:
df["pais2"] = df["pais"]
df["pais2"] = df["pais2"].fillna(sMode_Of_pais)
df

,idade,salario,pais,idade2,idade3,pais2
0,32.0,High,Spain,32.0,32.0,Spain
1,38.0,High,France,38.0,38.0,France
2,NaN,High,France,NaN,36.0,France
3,37.0,Low,NaN,37.0,37.0,France
4,NaN,Low,Germany,NaN,36.0,Germany
5,36.0,High,France,36.0,36.0,France
6,38.0,NaN,Spain,38.0,38.0,Spain
7,32.0,Medium,France,32.0,32.0,France
8,0.0,Medium,NaN,36.0,36.0,France
9,NaN,High,Spain,NaN,36.0,Spain


In [23]:
df["pais3"] = df["pais"].fillna('pais_mv')

In [24]:
df

,idade,salario,pais,idade2,idade3,pais2,pais3
0,32.0,High,Spain,32.0,32.0,Spain,Spain
1,38.0,High,France,38.0,38.0,France,France
2,NaN,High,France,NaN,36.0,France,France
3,37.0,Low,NaN,37.0,37.0,France,pais_mv
4,NaN,Low,Germany,NaN,36.0,Germany,Germany
5,36.0,High,France,36.0,36.0,France,France
6,38.0,NaN,Spain,38.0,38.0,Spain,Spain
7,32.0,Medium,France,32.0,32.0,France,France
8,0.0,Medium,NaN,36.0,36.0,France,pais_mv
9,NaN,High,Spain,NaN,36.0,Spain,Spain


# **EXERCÍCIOS**


## Exercício 1
* Trate os NaN's da variável 'salario'.

## Exercício 2 - Diabetes
* Carregue o dataframe diabeletes.csv e trate os NaN's.

### Carregar o dataframe

In [25]:
url_df= ''
df = pd.read_csv(url_df)
df.head()

FileNotFoundError: ignored

**Dica**: Algumas medidas não fazem sentido seram nulas (0). Portanto, os NaN's aqui neste dataframe são o valor 0. Portanto, substitua os NaN's (no caso, 0)das variáveis Glucose, BloodPressure, SkinThickness, Insulin e BMI por alguma medida como, por exemplo, média, mediana, moda e etc.

## Exercício 3 - Titanic
> Trate os NaN's do dataframe Titanic_With_MV.csv.